In [1]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy import integrate
from scipy.integrate import solve_ivp

In [2]:
plt.rc('text', usetex = True)
plt.rc('font', **{'family' : "sans-serif"})
params = {'text.latex.preamble' : [r'\usepackage{amsmath}']}
plt.rcParams.update(params)
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8

In [3]:
# Definiendo sistema de ecuaciones
def funciones(xi,y, alpha, n, h, Upsilon, varkappa):
    psi_ , eta_, psi1_, eta1_ = y
    dydxi = [-h*(eta_+ xi**(3)*(Upsilon*(psi_**(n+1) - varkappa**(1 + 1/n)) + alpha*(psi_**(n) - varkappa)))*(1 + Upsilon*(psi_ - (varkappa**(1 + 1/n)/psi_**(n))) + alpha*(1 - (varkappa/psi_**(n))))/xi/(xi - 2*Upsilon*(n+1)*eta_) /(1 + alpha*n/Upsilon/(n+1)/psi_) 
            ,xi**(2) * psi_**(n)
            ,-(h/xi/(xi - 2*Upsilon*(n+1)*eta_)/(1 + alpha*n/Upsilon/(n+1)/psi_))*((eta1_ + 3*xi**(2)*(Upsilon*(psi_**(n+1) - varkappa**(1 + 1/n)) + alpha*(psi_**(n) - varkappa)) + xi**(3)*(n+1)*Upsilon*psi_**(n)*psi1_*(1 + alpha*n/Upsilon/(n+1)/psi_))*(1 + Upsilon*(psi_ - varkappa**(1 + 1/n)/psi_**(n)) + alpha*(1 - varkappa/psi_**(n))) + (eta_ + xi**(3)*(Upsilon*(psi_**(n+1) - varkappa**(1+ 1/n)) + alpha*(psi_**(n) - varkappa)))*((n+1)*Upsilon*psi1_*(1 + alpha*n/Upsilon/(n+1)/psi_) - n*(Upsilon*(psi_**(n+1) - varkappa**(1 + 1/n)) + alpha*(psi_**(n) - varkappa))*psi1_/psi_**(n+1)) + (eta_ + xi**(3)*(Upsilon*(psi_**(n+1) - varkappa**(1 + 1/n)) + alpha*(psi_**(n) - varkappa)))*(1 + Upsilon*(psi_ - varkappa**(1 + 1/n)/psi_**(n)) + alpha*(1 - varkappa/psi_**(n)))*((2*Upsilon*(n+1)*eta1_ - 1)/(xi - 2*Upsilon*(n+1)*eta_) + alpha*n*psi1_/(1 + alpha*n/Upsilon/(n+1)/psi_)/Upsilon/(n+1)/psi_**(2) - 1/xi))
            ,2*xi*psi_**(n) + n*xi**(2)*psi_**(n-1)*psi1_] 
    return dydxi

In [4]:
c = 2.997*10**(8)          # Velocidad de la luz en S.I.
G = 6.673*10**(-11)        # Constante de gravitación universal en S.I.
Msun = 1.989*10**(30)      # Masa solar en S.I.

In [5]:
C = 0.0  # Valor de C para anisotropía

h = 1 - 2*C

alpha = 0.0 # Constante del término lineal

varkappa = 0.0 # Razón entre densidad en el borde y densidad central

Lista_n = [0.5,0.99999,1.5,2.0,2.5,2.99999,3.5,4.0]

print('Valores de n: ',end='')                                                    
print(*Lista_n, sep=', ')

Lista_sigma = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

print('Valores de \u03c3: ',end='')                                                    
print(*Lista_sigma, sep=', ')

Valores de n: 0.5, 0.99999, 1.5, 2.0, 2.5, 2.99999, 3.5, 4.0
Valores de σ: 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9


In [6]:
ListaModelos = []                                        # Container for all models 


for i in range(len(Lista_n)):
    
    ListaModelos.append([])                              # Container for C

In [7]:
%%time

Listaxi1 = []
ListaEta1 = []

for i in range(len(Lista_n)):
    
    n = Lista_n[i]
    
    for j in range(len(Lista_sigma)):
        
        sigma = Lista_sigma[j]
        
        Upsilon = (sigma - alpha*(1 - varkappa))/(1 - varkappa**(1 + 1/n))
        
        # Condiciones iniciales
        Psi0 = 1.0
        Eta0 = 0.0
        Psi10 = 0.0
        Eta10 = 0.0
        
        y0 = [Psi0,Eta0,Psi10,Eta10]
        
        def stop_condition(xi,y,alpha, n, h, Upsilon, varkappa):
            return (1/sigma)*(Upsilon*(y[0]**(n+1) - varkappa**(1 + 1/n)) + alpha*(y[0]**(n) - varkappa)) - 10**-16
        
        
        stop_condition.terminal = True
        
        xi0 = 10**(-14)
        ximax = 10**(11)
        
        xi_span = (xi0,ximax)
       
        
        soluciones = solve_ivp(funciones,xi_span,y0,method='RK45',events=stop_condition,
                               args=(alpha, n, h, Upsilon, varkappa),atol=10**-16)
        
        Psi = soluciones.y[0]
        Eta = soluciones.y[1]
        Psi1 = soluciones.y[2]
        Eta1 = soluciones.y[3]
        xi = soluciones.t
        
        Listaxi1.append(np.around(xi[-1],7))
        ListaEta1.append(np.around(Eta[-1],7))

C:\Users\DANIEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.
C:\Users\DANIEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
C:\Users\DANIEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  
C:\Users\DANIEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys


Wall time: 1.33 s


In [8]:
print(Listaxi1)
print(ListaEta1)

[2.2896375, 2.0002204, 1.8010134, 1.6541752, 1.540726, 1.4501277, 1.3756719, 1.3132584, 1.2600492, 2.5990533, 2.277009, 2.0642173, 1.913278, 1.800743, 1.7141068, 1.6460912, 1.5906602, 1.5450785, 3.0388682, 2.699673, 2.4938352, 2.3623382, 2.2758505, 2.2202434, 2.1859141, 2.1671415, 2.1600575, 3.7011015, 3.4021697, 3.2742363, 3.2503696, 3.2981002, 3.4009005, 3.5484753, 3.7350571, 3.9549109, 4.7852138, 4.7274717, 4.9900448, 5.5474284, 6.4337204, 7.724089, 9.5122894, 11.8732575, 14.7987621, 6.8318282, 7.958278, 10.8248755, 17.7550648, 36.8348623, 89.0045311, 157.4164195, 181.5491194, 181.9758042, 11.8812118, 23.5670834, 322.0700028, 5766.5322559, 2060.3227447, 1317.8512103, 1112.6242961, 1076.1829226, 1122.3716179, 38.9618234, 10987.8001772, 9016.0526139, 11740.4837123, 19888.179046, 31003.5551263, 35491.4187805, 35690.9548421, 34637.6772206]
[2.1743331, 1.4374827, 1.0360421, 0.7912222, 0.6296335, 0.5164131, 0.4341043, 0.371735, 0.3232059, 1.7516821, 1.1430276, 0.8193368, 0.6250091, 0.4981

In [9]:
pd.options.display.float_format = "{:,.7f}".format

In [10]:
data = pd.read_csv('Validación - Datos Dallas y Geroyannis (1993)',names = ['N', 'Sigma', 'Xi_b', 'Eta_b'],
                  engine='python')

In [11]:
data

,N,Sigma,Xi_b,Eta_b
0,0.5000000,0.1000000,2.2899149,2.1743665
1,0.5000000,0.2000000,2.0008463,1.4374281
2,0.5000000,0.3000000,1.8013545,1.0360530
3,0.5000000,0.4000000,1.6543434,0.7912060
4,0.5000000,0.5000000,1.5408662,0.6296190
5,0.5000000,0.6000000,1.4502055,0.5166507
6,0.5000000,0.7000000,1.3758251,0.4341211
7,0.5000000,0.8000000,1.3135016,0.3717053
8,0.5000000,0.9000000,1.2603785,0.3231648
9,1.0000000,0.1000000,2.5990904,1.7514303


In [12]:
data['Xib'] = Listaxi1
data['Etab'] = ListaEta1
data

,N,Sigma,Xi_b,Eta_b,Xib,Etab
0,0.5000000,0.1000000,2.2899149,2.1743665,2.2896375,2.1743331
1,0.5000000,0.2000000,2.0008463,1.4374281,2.0002204,1.4374827
2,0.5000000,0.3000000,1.8013545,1.0360530,1.8010134,1.0360421
3,0.5000000,0.4000000,1.6543434,0.7912060,1.6541752,0.7912222
4,0.5000000,0.5000000,1.5408662,0.6296190,1.5407260,0.6296335
5,0.5000000,0.6000000,1.4502055,0.5166507,1.4501277,0.5164131
6,0.5000000,0.7000000,1.3758251,0.4341211,1.3756719,0.4341043
7,0.5000000,0.8000000,1.3135016,0.3717053,1.3132584,0.3717350
8,0.5000000,0.9000000,1.2603785,0.3231648,1.2600492,0.3232059
9,1.0000000,0.1000000,2.5990904,1.7514303,2.5990533,1.7516821


In [14]:
distanciaXi = (abs(data['Xi_b']**2 - data['Xib']**2))**(1/2)
distanciaEta = (abs(data['Eta_b']**2 - data['Etab']**2))**(1/2)
data['Distancia Xi_b'] = distanciaXi
data['Distancia Eta_b'] = distanciaEta

errorXi = (abs(data['Xi_b'] - data['Xib'])/data['Xi_b'])*100
errorEta = (abs(data['Eta_b'] - data['Etab'])/data['Eta_b'])*100
data['Error Xi_b'] = errorXi
data['Error Eta_b'] = errorEta

pd.set_option("display.max_rows", None, "display.max_columns", None)

data

,N,Sigma,Xi_b,Eta_b,Xib,Etab,Distancia Xi_b,Distancia Eta_b,Error Xi_b,Error Eta_b
0,0.5000000,0.1000000,2.2899149,2.1743665,2.2896375,2.1743331,0.0356422,0.0120518,0.0121140,0.0015361
1,0.5000000,0.2000000,2.0008463,1.4374281,2.0002204,1.4374827,0.0500427,0.0125288,0.0312818,0.0037985
2,0.5000000,0.3000000,1.8013545,1.0360530,1.8010134,1.0360421,0.0350538,0.0047525,0.0189358,0.0010521
3,0.5000000,0.4000000,1.6543434,0.7912060,1.6541752,0.7912222,0.0235901,0.0050647,0.0101672,0.0020488
4,0.5000000,0.5000000,1.5408662,0.6296190,1.5407260,0.6296335,0.0207856,0.0042672,0.0090988,0.0022966
5,0.5000000,0.6000000,1.4502055,0.5166507,1.4501277,0.5164131,0.0150215,0.0156670,0.0053648,0.0459885
6,0.5000000,0.7000000,1.3758251,0.4341211,1.3756719,0.4341043,0.0205312,0.0038215,0.0111351,0.0038745
7,0.5000000,0.8000000,1.3135016,0.3717053,1.3132584,0.3717350,0.0252750,0.0046997,0.0185154,0.0079929
8,0.5000000,0.9000000,1.2603785,0.3231648,1.2600492,0.3232059,0.0288093,0.0051548,0.0261271,0.0127211
9,1.0000000,0.1000000,2.5990904,1.7514303,2.5990533,1.7516821,0.0138871,0.0296999,0.0014274,0.0143768


In [15]:
data[['N','Sigma','Error Xi_b','Error Eta_b']]

,N,Sigma,Error Xi_b,Error Eta_b
0,0.5000000,0.1000000,0.0121140,0.0015361
1,0.5000000,0.2000000,0.0312818,0.0037985
2,0.5000000,0.3000000,0.0189358,0.0010521
3,0.5000000,0.4000000,0.0101672,0.0020488
4,0.5000000,0.5000000,0.0090988,0.0022966
5,0.5000000,0.6000000,0.0053648,0.0459885
6,0.5000000,0.7000000,0.0111351,0.0038745
7,0.5000000,0.8000000,0.0185154,0.0079929
8,0.5000000,0.9000000,0.0261271,0.0127211
9,1.0000000,0.1000000,0.0014274,0.0143768
